In [1]:
import sys

sys.path.append("../..")

import nivapy3 as nivapy
import pandas as pd
import teotil2 as teo
import useful_rid_code as rid

# Generate overall summary table - 2022

Table 3 of the 2015 report summarises all monitored and modelled loads for the whole of Norway and for the four OSPAR regions. This table no longer appears in recent reports, but I find it to be a useful summary document. This notebook reads the output from the data processing so far and generates a Word version of Table 3. The code is based on the exploratory work [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/word_data_tables.ipynb).

In [2]:
# Year of interest
year = 2022

## 1. Summarise monitoring data

### 1.1. Identify network nodes with monitoring data

In [3]:
# Read station data
in_xlsx = r"/home/jovyan/shared/common/JES/teotil2_data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df.drop_duplicates(subset=["ospar_region", "nve_vassdrag_nr"])

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network
ann_input_csv = (
    f"/home/jovyan/shared/common/JES/teotil2/data/norway_annual_input_data/input_data_{year}.csv"
)
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

### 1.2. Read monitored fluxes

In [4]:
# Read data
in_csv = f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/measured_loads/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)

# Get just the downstream catchments
# mon_df = mon_df[mon_df['station_id'].isin(stn_df['station_id'].values)]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "new_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("new_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "new_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"]

# Rename cols to match template
mon_df["Flow rate_1000m3/day"] = mon_df["mean_q_1000m3/day"]
del mon_df["mean_q_1000m3/day"]

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

/tmp/ipykernel_1176/866753955.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mon_df1 = mon_df.groupby(["ospar_region", "new_rid_group"]).sum()
/tmp/ipykernel_1176/866753955.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mon_df2 = mon_df.groupby("new_rid_group").sum().reset_index()


Ag    As   Cd    Cr    Cu    Hg   NH4-N  \
ospar_region        new_rid_group                                             
LOFOTEN-BARENTS SEA rid_135        0.0   1.0  0.0   1.0   8.0   7.0   101.0   
                    rid_20         0.0   2.0  0.0   6.0  14.0  17.0   163.0   
NORTH SEA           rid_135        0.0   2.0  0.0   3.0  15.0  21.0   230.0   
                    rid_20         0.0   0.0  0.0   0.0   2.0   5.0    39.0   
NORWEGIAN SEA2      rid_135        0.0   5.0  0.0  11.0  35.0  35.0   381.0   
                    rid_20         0.0   1.0  0.0   4.0  17.0  15.0    35.0   
SKAGERAK            rid_135        0.0   2.0  0.0   1.0   6.0   9.0   136.0   
                    rid_20         0.0   5.0  0.0   9.0  44.0  52.0   790.0   
NORWAY              rid_135        0.0  10.0  1.0  16.0  65.0  72.0   848.0   
                    rid_20         0.0   9.0  1.0  19.0  77.0  90.0  1027.0   

                                     NO3-N    Ni  PO4-P    Pb       SPM  \
ospar_region        new_rid_group                                         
LOFOTEN-BARENTS SEA rid_135          345.0   6.0   18.0   0.0   17640.0   
                    rid_20           782.0  26.0  111.0   1.0  343001.0   
NORTH SEA           rid_135         4664.0   8.0   56.0   5.0   49530.0   
                    rid_20           844.0   1.0   10.0   0.0    6202.0   
NORWEGIAN SEA2      rid_135         3279.0  25.0  115.0   3.0  149064.0   
                    rid_20          1131.0   8.0   35.0   1.0   51722.0   
SKAGERAK            rid_135         1898.0   4.0   21.0   3.0   18322.0   
                    rid_20         11217.0  20.0  209.0   6.0  230336.0   
NORWAY              rid_135        10186.0  43.0  210.0  11.0  234556.0   
                    rid_20         13973.0  55.0  365.0   8.0  631261.0   

                                       SiO2       TOC     TOTN   TOTP     Zn  \
ospar_region        new_rid_group                                              
LOFOTEN-BARENTS SEA rid_135         27552.0   31769.0   1565.0   32.0    8.0   
                    rid_20         110627.0   67773.0   2951.0  209.0   18.0   
NORTH SEA           rid_135         43888.0   50296.0   8167.0  159.0   55.0   
                    rid_20           6570.0    8783.0   1275.0   32.0   10.0   
NORWEGIAN SEA2      rid_135         68295.0  105191.0   9718.0  264.0   83.0   
                    rid_20          30350.0   28598.0   2173.0   62.0   26.0   
SKAGERAK            rid_135         17925.0   43447.0   3926.0   82.0   38.0   
                    rid_20         131040.0  154499.0  17690.0  441.0  106.0   
NORWAY              rid_135        157661.0  230703.0  23375.0  537.0  185.0   
                    rid_20         278587.0  259654.0  24088.0  743.0  160.0   

                                   Flow rate  
ospar_region        new_rid_group             
LOFOTEN-BARENTS SEA rid_135          39947.0  
                    rid_20           69934.0  
NORTH SEA           rid_135         100623.0  
                    rid_20           14888.0  
NORWEGIAN SEA2      rid_135         139685.0  
                    rid_20           41653.0  
SKAGERAK            rid_135          24371.0  
                    rid_20          113496.0  
NORWAY              rid_135         304626.0  
                    rid_20          239971.0

## 2. Data for unmonitored areas

In [5]:
# Read data
in_csv = f"/home/jovyan/shared/common/JES/teotil2_data/results/unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,424088.0,13169.0,1097.0,3006.0,285.0,68453.0,11825.0,35796.0,737.0,658.0,...,14740.0,0.0,0.0,0.0,4.0,12.0,2.0,0.0,2.0,1306.0
LOFOTEN-BARENTS SEA,131069.0,1496.0,163.0,678.0,84.0,18092.0,3134.0,6051.0,117.0,98.0,...,2240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,346.0
NORTH SEA,114941.0,3556.0,460.0,494.0,70.0,21118.0,3640.0,12828.0,209.0,276.0,...,5991.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,402.0
NORWEGIAN SEA2,170790.0,2884.0,388.0,939.0,95.0,29220.0,5046.0,14083.0,317.0,233.0,...,4647.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,557.0
SKAGERAK,7289.0,5232.0,87.0,896.0,36.0,23.0,4.0,2834.0,94.0,52.0,...,1862.0,0.0,0.0,0.0,2.0,7.0,1.0,0.0,1.0,0.0


## 3. Create table

The cell below uses the 3-fold division of sites (`'rid_11'` etc.) from the old progreamme structure. To use this, copy the old Word template and make sure that the aggregation in Section 1.2 is done by `'old_rid_group'`.

In [ ]:
## Create table 3 (old programme structure)
# in_docx = f'../../../../Results/Word_Tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx'
#
# rid.write_word_overall_table(mon_df, umon_df, in_docx)

The cell below uses the 2-fold division of sites (`'rid_20'` and `'rid_135'`) for the 2017-20 progreamme structure. To use this, copy the new Word template and make sure that the aggregation in Section 1.2 is done by `'new_rid_group'`.

In [8]:
# Create table 3
in_docx = (
    f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/word_tables/{year + 1}Analysis_{year}Data/Table_3_{year}.docx"
)
rid.write_word_overall_table_2017_20(mon_df, umon_df, in_docx)

Finished.
